# Code for Binary Equlibrium Optimizer with Simulated Annealing (version 1.2.1)
## Last updated: 16/07/2020
### Coders: Ritam Guha, Kushal Kanti Ghosh


In [2]:
import numpy as np
import pandas as pd
import random
import math,time,sys
import openpyxl as xl
# from plot_graphs import plotter
from datetime import datetime
# from ipynb.fs.full.utilities import Ufunction, sign_func, onecount, reshape_np, compute_accuracy

# for knapsack problem change fitness_FS to fitness_Knapsack
# from ipynb.fs.full.utilities import fitness_Knapsack as fitness

In [6]:
def Ufunction(ip):
    alpha = 2
    beta = 1.5
    op = alpha * pow(abs(ip), beta)
    return op

def Vfunction(ip):
    op = 1 + ip*ip
    op = math.sqrt(op)
    op = ip/op
    return abs(op)

def sigmoid(ip):     
    if ip < 0:
        return 1 - 1/(1 + math.exp(ip))
    else:
        return 1/(1 + math.exp(-ip))


def sign_func(x): 
    if x<0:
        return -1
    return 1

def onecount(particle):
    return int(np.sum(particle))

def reshape_np(particle):
    return np.reshape(particle, (1, particle.shape[0]))

def compute_accuracy(train_X, train_Y, test_X, test_Y, particle):    
    cols=np.flatnonzero(particle)     
    if(cols.shape[0]==0):
        return 0    
    clf=KNeighborsClassifier(n_neighbors=5)
    train_data=train_X[:,cols]
    test_data=test_X[:,cols]
    clf.fit(train_data,train_Y)
    val=clf.score(test_data,test_Y)
    return val

def fitness_FS(particles):
    
    # fitness computation
    if(particles.ndim == 1):
        particles = reshape_np(particles)
        
    [num_particles, dimension] = particles.shape
    values = np.zeros((1, num_particles))
    count = 0
    
    for particle in particles:  
        current_particle = reshape_np(particle)
        set_cnt=int(np.sum(current_particle))
        
        if(set_cnt == 0):
            val = np.float64(1.0)     
            
        else:            
            set_cnt=set_cnt/dimension
            err = 1- compute_accuracy(train_X, train_Y, test_X, test_Y, particle)            
            val = omega*err + (1-omega)*set_cnt
        values[0, count] = np.float64(val) 
        count += 1            
    return values


def fitness_Knapsack(particles):
     # fitness computation
    if(particles.ndim == 1):
        particles = reshape_np(particles)
        
    [num_particles, dimension] = particles.shape
    values = np.zeros((1, num_particles))
    count = 0
    
    for particle in particles:  
        current_particle = reshape_np(particle)
        set_cnt=int(np.sum(current_particle))
        
        if(set_cnt == 0):
            val = np.float64(1.0)     
            
        else:            
            pos = np.flatnonzero(particle)                
            val = (np.sum(weights[pos])<=max_weight) * np.sum(worth[pos])
        values[0, count] = -np.float64(val) 
        count += 1            
    return values


def avg_concentration(eq_pool, pool_size, dimension):    
    avg = np.sum(eq_pool[0:pool_size,:], axis=0)         
    avg = avg/pool_size
    return avg


def neighbor(particle, percent=0.3):   
    current_particle = particle.copy()    
    dimension = current_particle.shape[1]
    num_change = int(dimension*percent)
    pos = np.random.randint(0,dimension-1,num_change)
    current_particle[0, pos] = 1 - current_particle[0, pos]
    return current_particle    

def initialize(partCount,dimension):
    population=np.zeros((partCount,dimension))
    minn = 1
    maxx = math.floor(0.5*dimension)
    if maxx<minn:
        maxx = minn + 1

    for i in range(partCount):
        random.seed(i**3 + 10 + time.time() ) 
        no = random.randint(minn,maxx)
        if no == 0:
            no = 1
        random.seed(time.time()+ 100)
        pos = random.sample(range(0,dimension-1),no)
        for j in pos:
            population[i][j]=1
        
    return population

In [4]:
def SA(population, fitness_list):
    
    # simulated annealing
    [particle_count, dimension] = np.shape(population)
    T0 = dimension

    for particle_no in range(particle_count):
        T=2*dimension
        current_particle = reshape_np(population[particle_no].copy())  
        current_fitness = fitness(current_particle) 
        best_particle = current_particle.copy()
        best_fitness = current_fitness.copy()        
        
        while T>T0:
            new_particle = neighbor(current_particle)
            new_fitness = np.float64(fitness(new_particle))            
            
            if new_fitness<best_fitness:
                current_particle=new_particle.copy()
                current_fitness=new_fitness.copy()
                best_particle=current_particle.copy()
                best_fitness=current_fitness.copy()
                
            elif new_fitness==best_fitness:
                if onecount(new_particle)<onecount(best_particle):
                    current_particle=new_particle.copy()
                    current_fitness=new_fitness.copy()
                    best_particle=current_particle.copy()
                    best_fitness=current_fitness.copy()
            else:            
                prob=np.exp((best_fitness-current_fitness)/T)
                if(random.random()<=prob):
                    current_particle=new_particle.copy()
                current_fitness=new_fitness
            T=int(T*0.7)                
        
        population[particle_no, :]=best_particle.copy() 
        fitness_list[0, particle_no]=best_fitness.copy()                   
        

In [5]:
def EO_SA(init_population, dimension, pool_size=4, max_iter=50, particle_count=100, allow_SA=True):
    
    # pool initialization
    population = init_population.copy()
    eq_pool = np.zeros((pool_size+1, dimension))
    eq_fit = np.zeros((1, pool_size))
    eq_fit[0, :] = 100
    conv_plot = []
        
    # iterations start
    for iter in range(max_iter):        
        pop_new = np.zeros((particle_count, dimension))        
        fitness_list = fitness(population) 
        sorted_idx = np.argsort(fitness_list)
        population = population[sorted_idx, :].reshape((particle_count, dimension))
        fitness_list = fitness_list[0, sorted_idx]

        # replacements in the pool
        for i in range(particle_count):
            for j in range(pool_size):                 
                if fitness_list[0, i] <= eq_fit[0, j]:
                    eq_fit[0, j] = fitness_list[0, i].copy()
                    eq_pool[j, :] = population[i, :].copy()
                    break
        

        conv_plot.append(eq_fit[0, 0])
        best_particle = eq_pool[0,:]
                
        Cave = avg_concentration(eq_pool,pool_size,dimension)
        eq_pool[pool_size] = Cave.copy()

        t = (1 - (iter/max_iter))**(a2*iter/max_iter)
        
        for i in range(particle_count):
            
            # randomly choose one candidate from the equillibrium pool
            random.seed(time.time() + 100 + 0.02*i)
            inx = random.randint(0,pool_size)
            Ceq = np.array(eq_pool[inx])

            lambda_vec = np.zeros(np.shape(Ceq))
            r_vec = np.zeros(np.shape(Ceq))
            for j in range(dimension):
                random.seed(time.time() + 1.1)
                lambda_vec[j] = random.random()
                random.seed(time.time() + 10.01)
                r_vec[j] = random.random()

            F_vec = np.zeros(np.shape(Ceq))
            for j in range(dimension):
                x = -1*lambda_vec[j]*t 
                x = math.exp(x) - 1
                x = a1 * sign_func(r_vec[j] - 0.5) * x

            random.seed(time.time() + 200)
            r1 = random.random()
            random.seed(time.time() + 20)
            r2 = random.random()
            if r2 < GP:
                GCP = 0
            else:
                GCP = 0.5 * r1
            G0 = np.zeros(np.shape(Ceq))
            G = np.zeros(np.shape(Ceq))
            for j in range(dimension):
                G0[j] = GCP * (Ceq[j] - lambda_vec[j]*population[i][j])
                G[j] = G0[j]*F_vec[j]
            
            # use transfer function to map continuous->binary
            for j in range(dimension):
                temp = Ceq[j] + (population[i][j] - Ceq[j])*F_vec[j] + G[j]*(1 - F_vec[j])/lambda_vec[j]                
                temp = Vfunction(temp)                
                if temp>np.random.random():
                    pop_new[i][j] = 1 - population[i][j]
                else:
                    pop_new[i][j] = population[i][j]                
        
        # performing simulated annealing
        if(allow_SA):            
            SA(pop_new, fitness_list)
    
            
        population = pop_new.copy()        
    
        
        
    return best_particle, conv_plot

In [15]:
############################################################################################################
# datasets=["BreastCancer","BreastEW","CongressEW","Exactly","Exactly2","HeartEW","Ionosphere","KrVsKpEW","Lymphography","M-of-n","PenglungEW","Sonar","SpectEW","Tic-tac-toe","Vote","WaveformEW","Wine","Zoo"]
def BEO_driver(dataset):

    # initializing components
    conv_plot_all = {}
    conv_plot = {}
    best_pop = -1
    best_iter = -1
    best_accuracy = -1
    best_no_features = -1      

    whole_accuracy = compute_accuracy(train_X, train_Y, test_X, test_Y, np.ones(dimension))
    print("Total Acc: ",whole_accuracy * 100)     

    # dictionary to store the final results
    final_result ={}    

    for particle_count in particle_count_testing:                

        for max_iter in max_iter_testing:            
            print("Particle count:", particle_count)
            print("Max iter:", max_iter)            
            
            accuracy_BEO = 0
            accuracy_BEOSA = 0
            best_accuracy_BEO = 0
            best_accuracy_BEOSA = 0

            num_features_BEO = 0
            num_features_BEOSA = 0   
            best_num_features_BEO = 0
            best_num_features_BEOSA = 0

            BEO_key = dataset + "_BEO" + "_pop_" + str(particle_count) + "_iter_" + str(max_iter)
            BEOSA_key = dataset + "_BEOSA" + "_pop_" + str(particle_count) + "_iter_" + str(max_iter)      

            for run in range(max_runs):
                
                print("Run", run)
                
                population = initialize(particle_count, dimension) 
                print("BEO starts...")
                best_particle_BEO, conv_BEO = EO_SA(init_population=population.copy(), pool_size=pool_size, max_iter=max_iter, particle_count=particle_count, dimension=dimension, allow_SA=False)            
                
                print("BEOSA starts...")
                best_particle_BEOSA, conv_BEOSA = EO_SA(init_population=population.copy(), pool_size=pool_size, max_iter=max_iter, particle_count=particle_count, dimension=dimension, allow_SA=True)                                                    

                conv_plot_all["BEO_pop_" + str(particle_count) + "_iter_" + str(max_iter)] = conv_BEO
                conv_plot_all["BEOSA_pop_" + str(particle_count)+ "_iter_" + str(max_iter)] = conv_BEOSA                        

                accuracy_BEO = compute_accuracy(train_X, train_Y, test_X, test_Y, best_particle_BEO) * 100
                accuracy_BEOSA = compute_accuracy(train_X, train_Y, test_X, test_Y, best_particle_BEOSA) * 100

                num_features_BEO = onecount(best_particle_BEO)
                num_features_BEOSA = onecount(best_particle_BEOSA)
                
                if(accuracy_BEO > best_accuracy_BEO):
                    best_accuracy_BEO = accuracy_BEO.copy()
                    best_num_features_BEO = np.float64(num_features_BEO).copy()
                    
                if(accuracy_BEOSA > best_accuracy_BEOSA):
                    best_accuracy_BEOSA = accuracy_BEOSA.copy()
                    best_num_features_BEOSA = np.float64(num_features_BEOSA).copy()

           
            if(best_accuracy_BEOSA > best_accuracy):
                best_pop = particle_count
                best_iter = max_iter                

            final_result[BEO_key + "_accuracy"] = best_accuracy_BEO
            final_result[BEOSA_key + "_accuracy"] = best_accuracy_BEOSA

            final_result[BEO_key + "_num_features"] = best_num_features_BEO
            final_result[BEOSA_key + "_num_features"] = best_num_features_BEOSA

            print('BEO:', best_accuracy_BEO)
            print('BEOSA:', best_accuracy_BEOSA)          


    print("Best combination:" + " Pop-" + str(best_pop) + " Iter-" + str(best_iter))
    conv_plot["BEO"] = conv_plot_all["BEO_pop_" + str(best_pop) + "_iter_" + str(max_iter)]
    conv_plot["BEOSA"] = conv_plot_all["BEOSA_pop_" + str(best_pop) + "_iter_" + str(max_iter)]
        
    return final_result, conv_plot    

In [16]:
# plottings 
def plot_graph(final_result, conv_plot, dataset):

    processes = ['BEO', 'BEOSA']   

    ################# plotting parameter-variation graphs ##################
    accuracy_var_pop_BEO = []
    accuracy_var_pop_BEOSA = []


    for particle_count in particle_count_testing:
        for max_iter in max_iter_testing:
            BEO_key = dataset + "_BEO" + "_pop_" + str(particle_count) + "_iter_" + str(max_iter) + "_runs_" + str(max_runs)
            BEOSA_key = dataset + "_BEOSA" + "_pop_" + str(particle_count) + "_iter_" + str(max_iter) + "_runs_" + str(max_runs)

            accuracy_var_pop_BEO.append(final_result[BEO_key + "_accuracy"])
            accuracy_var_pop_BEOSA.append(final_result[BEOSA_key + "_accuracy"])

    process_dict = {'BEO':accuracy_var_pop_BEO, 'BEOSA':accuracy_var_pop_BEOSA}
    output_file = dataset + "_parameter_variation" + ".png"
    plotter(dataset_name=dataset, x=particle_count_testing, x_label="Population size", y_dict=process_dict, y_objects=processes, y_label="Accuracy", title=dataset, storage_destination="Parameter Variation", file_name=output_file)        
    ###################################################################
    
    

    ################# plotting convergence graphs #####################
     
    x_range = np.arange(0, max_iter, 1)

    process_dict = {'BEO':conv_plot["BEO"], 'BEOSA':conv_plot["BEOSA"]}    
    output_file = dataset + "_convergence" + ".png"
    plotter(dataset_name=dataset, x=x_range, x_label="#Iteration", y_dict=process_dict, y_objects=processes, y_label="Fitness", title=dataset, storage_destination="Convergence", file_name=output_file)

    ###################################################################


In [17]:
def save_excel(final_result="", dataset="", save_type="parameter variation", init=0):
    # code to save the results in an excel file
    
    if(save_type == "parameter variation"):                
        
        if(init==1):     
            wb = xl.Workbook()
            ws = wb.active
            wb.remove(ws)
            for max_iter in max_iter_testing:            
                ws = wb.create_sheet("Iter_" + str(max_iter))
                ws.title = "Iter_" + str(max_iter)
                ws.merge_cells(start_row=1, start_column=1, end_row=3, end_column=1) 
                ws.cell(1,1).value = "population size"

                cur_row = 4
                cur_col = 1

                for particle_count in particle_count_testing:
                    ws.cell(cur_row, cur_col).value = particle_count
                    cur_row += 1

                
        else:
            wb = xl.load_workbook("Results/parameter_variation.xlsx")
            for max_iter in max_iter_testing:    
                ws = wb["Iter_" + str(max_iter)]
                cur_row = 1
                cur_col = ws.max_column + 1

                # setting headers
                ws.merge_cells(start_row=cur_row, start_column=cur_col, end_row=cur_row, end_column=cur_col+3) 
                ws.merge_cells(start_row=cur_row+1, start_column=cur_col, end_row=cur_row+1, end_column=cur_col+1)
                ws.merge_cells(start_row=cur_row+1, start_column=cur_col+2, end_row=cur_row+1, end_column=cur_col+3)

                ws.cell(cur_row, cur_col).value = dataset
                ws.cell(cur_row+1, cur_col).value = "BEO"
                ws.cell(cur_row+1, cur_col+2).value = "BEOSA"
                ws.cell(cur_row+2, cur_col).value = "Accuracy"
                ws.cell(cur_row+2, cur_col+1).value = "#features"
                ws.cell(cur_row+2, cur_col+2).value = "Accuracy"
                ws.cell(cur_row+2, cur_col+3).value = "#features"

                cur_row += 3            
                for particle_count in particle_count_testing:

                    BEO_key = dataset + "_BEO" + "_pop_" + str(particle_count) + "_iter_" + str(max_iter)
                    BEOSA_key = dataset + "_BEOSA" + "_pop_" + str(particle_count) + "_iter_" + str(max_iter)

                    ws.cell(cur_row, cur_col).value = final_result[BEO_key+"_accuracy"]
                    ws.cell(cur_row, cur_col+1).value = final_result[BEO_key+"_num_features"]
                    ws.cell(cur_row, cur_col+2).value = final_result[BEOSA_key+"_accuracy"]
                    ws.cell(cur_row, cur_col+3).value = final_result[BEOSA_key+"_num_features"]

                    cur_row += 1
                    
        wb.save("Results/parameter_variation.xlsx")   
                    
    elif(save_type == "convergence"):                

        if(init==1):     
            wb = xl.Workbook()
            ws = wb.active
            wb.remove(ws)
            
            for max_iter in max_iter_testing:            
                ws = wb.create_sheet("Iter_" + str(max_iter))
                ws.title = "Iter_" + str(max_iter)

                cur_row = 1
                cur_col = 1
                ws.merge_cells(start_row=cur_row, start_column=cur_col, end_row=cur_row+1, end_column=cur_col)
                ws.cell(cur_row, cur_col).value = "#Iteration"

                cur_row += 2

                for count in range(1, max_iter+1):
                    ws.cell(cur_row, cur_col).value = count
                    cur_row += 1


        else:
            wb = xl.load_workbook("Results/convergence.xlsx")
            for max_iter in max_iter_testing:    
                ws = wb["Iter_" + str(max_iter)]
                cur_row = 1
                cur_col = ws.max_column + 1

                # setting headers
                ws.merge_cells(start_row=cur_row, start_column=cur_col, end_row=cur_row, end_column=cur_col+1)                

                ws.cell(cur_row, cur_col).value = dataset
                ws.cell(cur_row+1, cur_col).value = "BEO"
                ws.cell(cur_row+1, cur_col+1).value = "BEOSA"                

                cur_row += 2            
                for iteration_no in range(max_iter):

                    conv_BEO = conv_plot["BEO"]
                    conv_BEOSA = conv_plot["BEOSA"]

                    ws.cell(cur_row, cur_col).value = conv_BEO[iteration_no]
                    ws.cell(cur_row, cur_col+1).value = conv_BEOSA[iteration_no]                    

                    cur_row += 1

        wb.save("Results/convergence.xlsx")    


### Set the global parameters and run the following cell to perform feature selection over UCI datasets

* This code will automatically create two excel files - one for parameter variation and another one for convergence. The results will be saved as parameter_variation and convergence in Results folder

* It will also plot convergence curves which will be saved in Convergence folder


In [100]:
# set the global parameters over here
datasets=["BreastCancer","BreastEW","CongressEW","Exactly","Exactly2","HeartEW","Ionosphere","KrVsKpEW","Lymphography","M-of-n","PenglungEW","Sonar","SpectEW","Tic-tac-toe","Vote","WaveformEW","Wine","Zoo"]
particle_count_testing = [10, 20]
max_iter_testing = [10, 20]
max_runs = 15
omega = 0.9                 
a2 = 1
a1 = 2
GP = 0.5
pool_size = 4
fitness = fitness_FS

# initializing the excel files
save_excel(init=1, save_type="parameter variation")
save_excel(init=1, save_type="convergence")

# start iterating over the datasets
for dataset in datasets:
    
    # reading dataset
    print("Dataset:", dataset)
    df=pd.read_csv("Data/"+dataset+".csv")
    (a,b)=np.shape(df)
    data = df.values[:,0:b-1]
    label = df.values[:,b-1]
    dimension = np.shape(data)[1] #particle dimension   
    print("dimension:", dimension)

    # loading_dataset
    cross = 5
    test_size = (1/cross)
    train_X, test_X, train_Y, test_Y = train_test_split(data, label,stratify=label ,test_size=test_size)
     

    final_result, conv_plot = BEO_driver(dataset)
    plot_graph(final_result, conv_plot, dataset)

    save_excel(final_result, dataset, save_type="parameter variation")
    save_excel(final_result, dataset, save_type="convergence")

Dataset: BreastCancer
dimension: 10
Total Acc:  57.85714285714286
Particle count: 10
Max iter: 10
Run 0
BEO starts...
BEOSA starts...


KeyboardInterrupt: 

### Run the following cell for solving knapsack problem

In [7]:
file_path = 'Data/Knapsack/'
file_names = ["ks_8a", "ks_8b", "ks_8c", "ks_8d", "ks_8e", "ks_12a", "ks_12b", "ks_12c", "ks_12d", "ks_12e", "ks_16a", "ks_16b", "ks_16c", "ks_16d", "ks_16e", "ks_20a", "ks_20b", "ks_20c", "ks_20d", "ks_20e", "ks_24a", "ks_24b", "ks_24c", "ks_24d", "ks_24e"]

# set the global parameters
omega = 0.9                 
a2 = 1
a1 = 2
GP = 0.5
max_runs = 30
particle_count = 20
dimension = -1
max_iter = 500
pool_size = 30
fitness = fitness_Knapsack

weights = []
worth = []
max_weight = -1

for file_name in file_names:    
    fname = file_path + file_name + ".txt"     
    x = open(fname, 'r')
    count = int(x.readline())
    
    weights = np.array(list(map(int, x.readline().split())))
    worth = np.array(list(map(int, x.readline().split())))
    max_weight = int(x.readline())

    dimension = count
    
    population = initialize(particle_count, dimension)
    fitness_list = fitness(population)

    sorted_idx = np.argsort(fitness_list)
    population = population[sorted_idx, :].reshape((particle_count, dimension))
    fitness_list = fitness_list[0, sorted_idx]

    best_particle_BEO = np.zeros((max_runs, dimension))
    best_particle_BEOSA = np.zeros((max_runs, dimension))
    
    conv_BEO = np.zeros((max_runs, max_iter))
    conv_BEOSA = np.zeros((max_runs, max_iter))
    
    best_fitness_BEO = np.zeros((1, max_runs))
    best_fitness_BEOSA = np.zeros((1, max_runs))

    for run_no in range(max_runs):    
        best_particle_BEO[run_no,:], conv_BEO[run_no,:] = EO_SA(init_population=population, dimension=dimension, pool_size=pool_size, max_iter=max_iter, particle_count=particle_count, allow_SA=False)
        best_particle_BEOSA[run_no,:], conv_BEOSA[run_no,:] = EO_SA(init_population=population, dimension=dimension, pool_size=pool_size, max_iter=max_iter, particle_count=particle_count, allow_SA=True)
    
    best_fitness_BEO[0, :] = fitness(best_particle_BEO)
    best_fitness_BEOSA[0, :] = fitness(best_particle_BEOSA)
    
    print(-np.mean(best_fitness_BEO), np.std(best_fitness_BEO), -np.mean(best_fitness_BEOSA), np.std(best_fitness_BEOSA))
    
#     print("Dataset:%s  Mean:%f  Std:%f" %(file_name, -np.mean(best_fitness), np.std(best_fitness)))

3610494.0 0.0 3924400.0 0.0
3740776.0 0.0 3740776.0 0.0
3347452.0 0.0 3347452.0 0.0
4082475.0 0.0 4082475.0 0.0
4856975.366666666 1355.0869709686126 4857227.0 0.0
5685907.7 16044.021509895829 5688887.0 0.0
6463514.0 0.0 6498597.0 0.0
5169676.0 0.0 5169676.0 0.0
6903672.0 0.0 6992404.0 0.0
5227215.0 0.0 5227215.0 0.0
7821255.0 0.0 7842384.5 11703.505138062985
8881820.0 0.0 9341431.3 19418.632957634614
8948033.866666667 81936.00056252578 9141289.3 14477.438371825316
9126071.0 0.0 9326519.033333333 22462.72019218114
7267954.0 0.0 7760444.8 8221.736395677011
10558912.166666666 21681.380632673947 10659381.0 31178.235206630925
9536420.7 27402.723153183157 9740663.0 33023.791986990225
10257609.0 0.0 10624542.633333333 44763.86923586427
8740236.966666667 7519.933605572739 8863593.933333334 31942.57243964897
9300655.9 10148.63206989001 9309090.733333332 18480.356587348513
13017232.633333333 17699.331415778266 13398986.3 50160.74151641567
11997124.166666666 38797.60952694838 12075179.066666666 3